In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="ifu")

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs

doc_dir = "haystackdata/ifu"
docs = convert_files_to_docs(dir_path=doc_dir, split_paragraphs=True)
print(docs[:3])
document_store.write_documents(docs)

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


In [ ]:
from haystack.utils import print_answers

# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="beam limitation", params={"Retriever": {"top_k": 50}, "Reader": {"top_k": 5}}
)

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="medium")
